In [1]:
import os
os.chdir('K:/who/html')

In [2]:
import pandas as pd

In [3]:
import datetime as dt
import numpy as np
import re

In [4]:
#read table from html
for i in os.listdir():

    f=open(i,'r',encoding='utf_8_sig')

    raw=f.read()

    f.close()
    
    try:
        df=pd.read_html(raw)
        name=pd.to_datetime(i.replace('.html','')).strftime('%Y-%m-%d')
        df[0].to_csv(f'K:/who/data/{name}.csv',index=False)
    except ValueError:
        
        #for some reason, august 2003 report comes from pdf
        #we have to take extra time to process it
        text="""Areas Female Male Total Median_age_(range) Number_of_cases_currently_hospitalised Number_of_cases_recovered Number_of_deaths CFR_(%) Number_of_imported_cases_(%) Number_of_HCW_affected_(%) Date_onset_first_probable_case Date_onset_last_probable_case
        Australia 4 2 6 15(1-45) 0 6 0 0 6(100) 0(0) 24-Mar-03 1-Apr-03
        Brazil 1 1 NA 4 0 1 0 0 1(100) 0(0) 3-Apr-03 3-Apr-03
        Canada 151 100 251 49(1-98) 10 200 41 17 5(2) 108(43) 23-Feb-03 12-Jun-03
        China Pending Pending 5327 Pending 29 4949 349 7 NA 1002(19) 16-Nov-02 25-Jun-03
        China,_Hong_Kong_Special_Administrative_Region 977 778 1755 40(0-100) 7 1448 300 17 NA 386(22) 15-Feb-03 31-May-03
        China,_Macao_Special_Administrative_Region 0 1 1 28 0 1 0 0 1(100) 0(0) 5-May-03 5-May-03
        China,_Taiwan 349 319 665 46(2-79) 10 475 180 27 50(8) 86(13) 25-Feb-03 15-Jun-03
        Colombia 1 0 1 28 0 1 0 0 1(100) 0(0) 2-Apr-03 2-Apr-03
        Finland 0 1 1 24 0 1 0 0 1(100) 0(0) 30-Apr-03 30-Apr-03
        France 1 6 7 49(26-61) 0 6 1 14 7(100) 2(29) 21-Mar-03 3-May-03
        Germany 4 5 9 44(4-73) 0 9 0 0 9(100) 1(11) 9-Mar-03 6-May-03
        India 0 3 3 25(25-30) 0 3 0 0 3(100) 0(0) 25-Apr-03 6-May-03
        Indonesia 0 2 2 56(47-65) 0 2 0 0 2(100) 0(0) 6-Apr-03 17-Apr-03
        Italy 1 3 4 30.5(25-54) 0 4 0 0 4(100) 0(0) 12-Mar-03 20-Apr-03
        Kuwait 1 0 1 50 0 1 0 0 1(100) 0(0) 9-Apr-03 9-Apr-03
        Malaysia 1 4 5 30(26-84) 0 3 2 40 5(100) 0(0) 14-Mar-03 22-Apr-03
        Mongolia 8 1 9 32(17-63) 0 9 0 0 8(89) 1(11) 31-Mar-03 6-May-03
        New_Zealand 1 0 1 67 0 1 0 0 1(100) NA 20-Apr-03 20-Apr-03
        Philippines 8 6 14 41(29-73) 0 12 2 14 7(50) 4(29) 25-Feb-03 5-May-03
        Republic_of_Ireland 0 1 1 56 0 1 0 0 1(100) 0(0) 27-Feb-03 27-Feb-03
        Republic_of_Korea 0 3 3 40(20-80) 0 3 0 0 3(100) 0(0) 25-Apr-03 10-May-03
        Romania 0 1 1 52 0 1 0 0 1(100) 0(0) 19-Mar-03 19-Mar-03
        Russian_Federation 0 1 1 25 1 0 0 NA NA 0(0) 5-May-03 5-May-03
        Singapore 161 77 238 35(1-90) 0 205 33 14 8(3) 97(41) 25-Feb-03 5-May-03
        South_Africa 0 1 1 62 0 0 1 100 1(100) 0(0) 3-Apr-03 3-Apr-03
        Spain 0 1 1 33 0 1 0 0 1(100) 0(0) 26-Mar-03 26-Mar-03
        Sweden 1 2 3 33 0 3 0 0 3(100) 0(0) NA NA
        Switzerland 0 1 1 35 0 1 0 0 1(100) 0(0) 9-Mar-03 9-Mar-03
        Thailand 5 4 9 42(2-79) 0 7 2 22 9(100) 1(11) 11-Mar-03 27-May-03
        United_Kingdom 2 2 4 59(28-74) 0 4 0 0 4(100) 0(0) 1-Mar-03 1-Apr-03
        United_States 16 17 33 36(0-83) 7 26 0 0 31(94) 1(3) 9-Jan-03 13-Jul-03
        Viet_Nam 39 24 63 43(20-76) 0 58 5 8 1(2) 36(57) 23-Feb-03 14-Apr-03
        Total NA NA 8422 NA 64 7442 916 11 NA 1725(20) NA NA"""

        text_process=[j for i in text.split('\n') for j in i.split(' ') if j]

        output=pd.DataFrame()
        for i in range(int(len(text_process)/13)):
            output[i]=(text_process[i*13:i*13+13])

        output=output.T

        output.columns=output.loc[0].str.replace('_',' ')

        output.drop(0,inplace=True)

        output=output.replace('NA','')

        output['Areas']=output['Areas'].str.replace('_',' ')

        output.to_csv('K:/who/data/2003-08-15.csv',index=False)

In [5]:
os.chdir('K:/who/data')

In [6]:
for i in os.listdir():
    df=pd.read_csv(i)
    
    #clean up useless columns
    for j in df.columns:
        if df[j].isnull().all():
            del df[j]
            
    #clean up useless rows
    if 'Unnamed: 0' in df.columns.tolist() or '0' in df.columns.tolist():
        df.columns=df.loc[0]
        df.drop(0,inplace=True)    
    
    df.to_csv(i,index=False)

In [7]:
#clean up some of the headers
for i in os.listdir()[21:33]:
    df=pd.read_csv(i)
    
    if len(df.columns.tolist())==6:
        df.columns=['Country', 'Cumulative number of cases',
           'Number of new cases since last WHO update', 'Number of deaths','Number recovered',
           'Local chains of transmission']
        df.drop(0,inplace=True)
    if len(df.columns.tolist())==7:
        df.columns=['Country', 'Cumulative number of cases',
           'Number of new cases since last WHO update', 'Number of deaths','Number recovered',
           'Local chains of transmission','Date of last report']
        df.drop(0,inplace=True)
    df.to_csv(i,index=False)

In [8]:
#standardize column name
#cant really use regex as i struggle to find pattern
#who should use better table format and fewer footnotes
D={'Cumulative number of case(s)': 'Cumulative number of cases',
 'Cumulative number of case(s) §': 'Cumulative number of cases',
 'Cumulative number of case(s)2': 'Cumulative number of cases',
 'Cumulative number of case(s)^1': 'Cumulative number of cases',
 'Cumulative number of case(s)^2': 'Cumulative number of cases',
 'Cumulative number of cases^2': 'Cumulative number of cases',
 'Total': 'Cumulative number of cases',
 'Total number of case(s)': 'Cumulative number of cases',
  'Local chain(s) of transmission 2': 'Local transmission',
 'Local chain(s) of transmission2': 'Local transmission',
 'Local chain(s) of transmission4': 'Local transmission',
 'Local chain(s) of transmission^3': 'Local transmission',
 'Local chain(s) of transmission^4': 'Local transmission',
 'Local chains of transmission': 'Local transmission',
 'Local transmission*': 'Local transmission',
 'Date last probable case reported': 'Date of last report',
 'Date of last report': 'Date of last report',
 'Date onset last probable case': 'Date of last report',
  'Areas':'Country',
  'CFR (%)': 'Case fatality ratio (%)',
  'Final Status/ Number of deaths': 'Number of deaths',
 'Final status/ Number of deaths': 'Number of deaths',
 'Number of deaths^a': 'Number of deaths',
  'Number of cases recovered': 'Number recovered',
 'Final Status/ Number recovered^2': 'Number recovered',
 'Final status/ Number recovered3': 'Number recovered',
 'Final status/ Number recovered^2': 'Number recovered',
 'Final status/ Number recovered^3': 'Number recovered',
 'Number recovered3': 'Number recovered',
 'Number recovered^3': 'Number recovered',
 'Number recovered^4': 'Number recovered',
  'Number of new cases since last WHO update 2, 3': 'Number of new cases since last WHO update',
 'Number of new cases since last WHO update2': 'Number of new cases since last WHO update',
 'Number of new cases since last WHO update2,3': 'Number of new cases since last WHO update',
 'Number of new cases since last WHO update2,^3': 'Number of new cases since last WHO update',
 'Number of new cases since last WHO update2.3': 'Number of new cases since last WHO update',
 'Number of new cases since last WHO update^2': 'Number of new cases since last WHO update'}

In [9]:
cols=[]
for i in os.listdir():
    df=pd.read_csv(i)
    df.columns=pd.Series(df.columns).replace(D)
    cols+=(df.columns.tolist())
    df.to_csv(i,index=False)

In [10]:
grand=pd.DataFrame(columns=list(set(cols))+['report date'])
for i in os.listdir():

    df=pd.read_csv(i)
    for j in set(cols):
        if j not in grand.columns:
            df[j]=''
            
    df['report date']=i.replace('.csv','')
    
    grand=grand.append(df,sort=False,ignore_index=True)

In [11]:
grand['Number of new cases since last WHO update']=grand['Number of new cases since last WHO update'].replace('not applicable',np.nan).apply(float)

In [12]:
#clean footnote from country
country_clean=[
'Australia',
 'Belgium',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Guangdong',
 'Hong Kong',
 'Macao',
 'Colombia',
 'Finland',
 'France',
 'Germany',
 'India',
 'Indonesia',
 'Italy',
 'Japan',
 'Kuwait',
 'Malaysia',
 'Mongolia',
 'New Zealand',
 'Philippines',
 'Poland',
 'Republic of Ireland',
 'Republic of Korea',
 'Romania',
 'Russian Federation',
 'Singapore',
 'Slovenia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Thailand',
 'Total',
 'United Kingdom',
 'United States',
 'Viet Nam']

country_raw=grand['Country'].tolist()

for i in range(len(country_raw)):
    for j in country_clean:
        if j in str(country_raw[i]):
            country_raw[i]=j
            
    if 'China' in str(country_raw[i]):
        country_raw[i]='China'

grand['Country']=country_raw

ind=grand.loc[grand['Country'].isna()].index

grand.drop(ind,inplace=True)

In [13]:
#clean footnote
grand['Date of last report']=pd.to_datetime(grand['Date of last report'].replace('13-Jul-03^e',''))

In [14]:
#clean footnote
grand['Number of deaths']=grand['Number of deaths'].replace(value='',regex='\*').apply(float)

In [15]:
#divide column into two and clean up
grand['Number of imported cases']=grand['Number of imported cases (%)'].apply(lambda x: re.findall('\d*',str(x).split('(')[0])[0])

grand['Number of imported cases']=grand['Number of imported cases'].replace('',np.nan).astype(float)

grand['percentage of imported cases']=grand['Number of imported cases (%)'].apply(lambda x: int(re.findall('(?<=\()\d+',x)[0]) if '(' in str(x) else np.nan)

del grand['Number of imported cases (%)']

In [16]:
#divide column into two and clean up
grand['Number of HCW affected']=grand['Number of HCW affected (%)'].apply(lambda x: re.findall('\d*',str(x).split('(')[0])[0])

grand['Number of HCW affected']=grand['Number of HCW affected'].replace('',np.nan).apply(float)

grand['percentage of HCW affected']=grand['Number of HCW affected (%)'].apply(lambda x: int(re.findall('(?<=\()\d+',x)[0]) if '(' in str(x) else np.nan)

del grand['Number of HCW affected (%)']

In [17]:
#few columns are squeaky clean
grand['Date onset first probable case']=pd.to_datetime(grand['Date onset first probable case'])
grand['Date for which cumulative number of cases is current']=pd.to_datetime(grand['Date for which cumulative number of cases is current'])
grand['report date']=pd.to_datetime(grand['report date'])

In [18]:
#remove useless info
grand['Male']=grand['Male'].replace('Pending',np.nan).astype(float)
grand['Female']=grand['Female'].replace('Pending',np.nan).astype(float)

In [19]:
#clean footnote
grand['Cumulative number of cases']=grand['Cumulative number of cases'].apply(lambda x:re.findall('\d*',str(x))[0])

grand['Cumulative number of cases']=grand['Cumulative number of cases'].replace('',np.nan).astype(float)

In [20]:
#remove useless info
grand['Number recovered']=grand['Number recovered'].replace('not available',np.nan).astype(float)

In [21]:
#divide column into tree and clean up
grand['median age']=grand['Median age (range)'].apply(lambda x: re.findall('\d*',str(x).split('(')[0])[0])

grand['median age']=grand['median age'].replace('',np.nan).apply(float)

grand['minimum age']=grand['Median age (range)'].apply(lambda x: int(re.findall('(?<=\()\d+',x)[0]) if '(' in str(x) else np.nan)

grand['maximum age']=grand['Median age (range)'].apply(lambda x: int(''.join(re.findall('\d*',x.split('-')[1]))) if '-' in str(x) else np.nan)

del grand['Median age (range)']

In [22]:
grand['Local transmission']=grand['Local transmission'].replace({'Being determined':np.nan,'To be determined':np.nan,'None*':'No','None':'No'})

In [23]:
grand.to_csv('K:/who/grand.csv',index=False)